<a href="https://colab.research.google.com/github/tawaqalt/arbritrary/blob/master/Tawakalitu_Yusuf_Keras_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Problem 1] Sharing and executing the official tutorial model

In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.15.0


In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11490434/11490434 [==============================] - 0s 0us/step


## Build a machine learning model

Build a `tf.keras.Sequential` model:

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

In [ ]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.40259278,  0.5827957 ,  0.10453623, -0.49031234, -0.13483012,
        -0.31236565,  0.3927142 , -0.19655225, -0.32125467,  0.24674672]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to *probabilities* for each class:

In [ ]:
tf.nn.softmax(predictions).numpy()

array([[0.13693613, 0.16397558, 0.10164212, 0.05607049, 0.08000521,
        0.06699086, 0.13559006, 0.07521643, 0.06639802, 0.117175  ]],
      dtype=float32)

Define a loss function for training using `losses.SparseCategoricalCrossentropy`:

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

The loss function takes a vector of ground truth values and a vector of logits and returns a scalar loss for each example. This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [ ]:
loss_fn(y_train[:1], predictions).numpy()

2.7031991

In [ ]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [ ]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - loss: 0.0698 - accuracy: 0.9790 - 629ms/epoch - 2ms/step


[0.06983572244644165, 0.9789999723434448]

In [ ]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [ ]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[2.90592972e-09, 1.83336935e-09, 1.15094053e-05, 6.73986142e-05,
        6.06224749e-12, 4.76958064e-08, 1.76434144e-13, 9.99920726e-01,
        5.84759192e-08, 2.56671797e-07],
       [6.29487296e-10, 3.33659205e-04, 9.99662161e-01, 2.53803069e-06,
        2.71335743e-13, 2.03424918e-07, 5.37389866e-09, 2.86747182e-14,
        1.38465293e-06, 6.52231632e-13],
       [1.71469722e-07, 9.99462426e-01, 7.17014191e-05, 9.68696804e-06,
        1.65514975e-05, 5.61148170e-07, 8.35000355e-06, 3.23058048e-04,
        1.07108914e-04, 3.98671403e-07],
       [9.98885691e-01, 2.33585027e-08, 4.02420555e-05, 3.86875081e-06,
        6.04595016e-06, 2.81772263e-05, 4.84819233e-04, 1.34536574e-04,
        6.43496833e-07, 4.16018796e-04],
       [5.48915250e-06, 1.58233817e-07, 3.47472778e-05, 2.52011796e-06,
        9.87487555e-01, 2.71242379e-06, 2.94105703e-05, 4.03273480e-05,
        3.44278328e-06, 1.23936189e-02]], dtype=float32)>

# [Problem 3] Learning Iris (binary classification) with Keras

In [14]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

# Load the Iris dataset
iris = pd.read_csv('Iris.csv')

# Filter for Iris-versicolor and Iris-virginica
iris_binary = iris[(iris['Species'] == 'Iris-versicolor') | (iris['Species'] == 'Iris-virginica')]

# Extract features and labels
X = iris_binary.drop('Species', axis=1)
y = iris_binary['Species']

# Encode labels (0 for Iris-versicolor, 1 for Iris-virginica)
y = np.where(y == 'Iris-versicolor', 0, 1)

# Split into training and testing sets
X_train_binary, X_test_binary, y_train_binary, y_test_binary = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_binary = scaler.fit_transform(X_train_binary)
X_test_binary = scaler.transform(X_test_binary)

# Define the model
model_binary = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu', input_shape=(X_train_binary.shape[1],)),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
loss_fn = tf.keras.losses.BinaryCrossentropy()
model_binary.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

# Train the model
model_binary.fit(X_train_binary, y_train_binary, epochs=5, batch_size=5, validation_split=0.2)

# Evaluate the model
loss, accuracy = model_binary.evaluate(X_test_binary, y_test_binary)
print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}')

# Make predictions
predictions_binary = model_binary.predict(X_test_binary)
predicted_classes_binary = (predictions_binary > 0.5).astype(int)
print(predicted_classes_binary.flatten())


Epoch 1/5
13/13 [==============================] - 1s 18ms/step - loss: 0.6786 - accuracy: 0.5312 - val_loss: 0.7021 - val_accuracy: 0.4375
Epoch 2/5
13/13 [==============================] - 0s 6ms/step - loss: 0.6413 - accuracy: 0.5312 - val_loss: 0.6708 - val_accuracy: 0.5000
Epoch 3/5
13/13 [==============================] - 0s 6ms/step - loss: 0.6044 - accuracy: 0.5781 - val_loss: 0.6437 - val_accuracy: 0.5000
Epoch 4/5
13/13 [==============================] - 0s 6ms/step - loss: 0.5735 - accuracy: 0.6875 - val_loss: 0.6186 - val_accuracy: 0.6250
Epoch 5/5
1/1 [==============================] - 0s 26ms/step - loss: 0.4206 - accuracy: 0.9000
Test Loss: 0.4206, Test Accuracy: 0.9000
1/1 [==============================] - 0s 89ms/step
[1 1 1 0 0 0 0 1 0 1 0 0 1 0 0 0 1 1 0 0]


# [Problem 4] Learning Iris (multi-level classification) with Keras

In [20]:
# Load the Iris dataset
iris = pd.read_csv('Iris.csv')

# Extract features and labels
X = iris.drop('Species', axis=1)
y = iris['Species']

# One-hot encode the labels
encoder = OneHotEncoder()
y_encoded = encoder.fit_transform(y.values.reshape(-1, 1)).toarray()

# Split into training and testing sets
X_train_multi, X_test_multi, y_train_multi, y_test_multi = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_multi = scaler.fit_transform(X_train_multi)
X_test_multi = scaler.transform(X_test_multi)

# Define the model
model_multi = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu', input_shape=(X_train_multi.shape[1],)),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

# Compile the model
loss_fn = tf.keras.losses.CategoricalCrossentropy()
model_multi.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

# Train the model
model_multi.fit(X_train_multi, y_train_multi, epochs=10, batch_size=5, validation_split=0.2)

# Evaluate the model
loss, accuracy = model_multi.evaluate(X_test_multi, y_test_multi)
print(f'\nTest Loss: {loss:.4f}, \nTest Accuracy: {accuracy:.4f}')

# Make predictions
predictions_multi = model_multi.predict(X_test_multi)
predicted_classes_multi = np.argmax(predictions_multi, axis=1)
print(predicted_classes_multi)


Epoch 1/10
20/20 [==============================] - 1s 12ms/step - loss: 1.0888 - accuracy: 0.2812 - val_loss: 1.0956 - val_accuracy: 0.1667
Epoch 2/10
20/20 [==============================] - 0s 4ms/step - loss: 1.0005 - accuracy: 0.3438 - val_loss: 1.0286 - val_accuracy: 0.2500
Epoch 3/10
20/20 [==============================] - 0s 4ms/step - loss: 0.9251 - accuracy: 0.5104 - val_loss: 0.9726 - val_accuracy: 0.2917
Epoch 4/10
20/20 [==============================] - 0s 4ms/step - loss: 0.8581 - accuracy: 0.6250 - val_loss: 0.9236 - val_accuracy: 0.4583
Epoch 5/10
20/20 [==============================] - 0s 7ms/step - loss: 0.8030 - accuracy: 0.6458 - val_loss: 0.8818 - val_accuracy: 0.5417
Epoch 6/10
20/20 [==============================] - 0s 5ms/step - loss: 0.7556 - accuracy: 0.7188 - val_loss: 0.8427 - val_accuracy: 0.8750
Epoch 7/10
20/20 [==============================] - 0s 6ms/step - loss: 0.7133 - accuracy: 0.8125 - val_loss: 0.8043 - val_accuracy: 0.9167
Epoch 8/10
20/20 [=


Test Loss: 0.5769, 
Test Accuracy: 1.0000
1/1 [==============================] - 0s 53ms/step
[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]


# [Problem 5] Learning House Prices with Keras

In [23]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

# Load the House Prices dataset
house_prices = pd.read_csv('train.csv')

# Extract features and labels
X = house_prices[['GrLivArea', 'YearBuilt']]
y = house_prices['SalePrice']

# Split into training and testing sets
X_train_hp, X_test_hp, y_train_hp, y_test_hp = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_hp = scaler.fit_transform(X_train_hp)
X_test_hp = scaler.transform(X_test_hp)

# Define the model
model_hp = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train_hp.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)  # Output layer for regression
])

# Compile the model
loss_fn = tf.keras.losses.MeanSquaredError()
model_hp.compile(optimizer='adam', loss=loss_fn, metrics=['MeanAbsoluteError'])

# Train the model
model_hp.fit(X_train_hp, y_train_hp, epochs=5, batch_size=32, validation_split=0.2)

# Evaluate the model
loss, mae = model_hp.evaluate(X_test_hp, y_test_hp)
print(f'\nTest Mean Absolute Error: ${mae:.2f}')

# Make predictions
predictions_hp = model_hp.predict(X_test_hp)
print(predictions_hp.flatten())


Epoch 1/5
30/30 [==============================] - 1s 8ms/step - loss: 39146848256.0000 - mean_absolute_error: 181526.1562 - val_loss: 37841874944.0000 - val_mean_absolute_error: 181101.3438
Epoch 2/5
30/30 [==============================] - 0s 4ms/step - loss: 39146401792.0000 - mean_absolute_error: 181524.9219 - val_loss: 37841203200.0000 - val_mean_absolute_error: 181099.5312
Epoch 3/5
30/30 [==============================] - 0s 3ms/step - loss: 39145373696.0000 - mean_absolute_error: 181522.1250 - val_loss: 37839720448.0000 - val_mean_absolute_error: 181095.4531
Epoch 4/5
30/30 [==============================] - 0s 3ms/step - loss: 39143251968.0000 - mean_absolute_error: 181516.3281 - val_loss: 37836787712.0000 - val_mean_absolute_error: 181087.5625
Epoch 5/5
10/10 [==============================] - 0s 2ms/step - loss: 39643316224.0000 - mean_absolute_error: 178810.6719

Test Mean Absolute Error: $178810.67
10/10 [==============================] - 0s 2ms/step
[26.938606 37.01614  3

# [Problem 6] Learning MNIST with Keras

In [24]:
from tensorflow.keras.datasets import mnist

# Load the MNIST dataset
(X_train_mnist, y_train_mnist), (X_test_mnist, y_test_mnist) = mnist.load_data()

# Normalize the images
X_train_mnist = X_train_mnist / 255.0
X_test_mnist = X_test_mnist / 255.0

# Reshape the images
X_train_mnist = X_train_mnist.reshape(X_train_mnist.shape[0], 28, 28, 1)
X_test_mnist = X_test_mnist.reshape(X_test_mnist.shape[0], 28, 28, 1)

# Define the model
model_mnist = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10)  # Output layer for 10 classes
])

# Compile the model
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model_mnist.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

# Train the model
model_mnist.fit(X_train_mnist, y_train_mnist, epochs=5, batch_size=32, validation_data=(X_test_mnist, y_test_mnist))

# Evaluate the model
loss, accuracy = model_mnist.evaluate(X_test_mnist, y_test_mnist)
print(f'\nTest Loss: {loss:.4f}, \nTest Accuracy: {accuracy:.4f}')

# Make predictions
predictions_mnist = model_mnist.predict(X_test_mnist)
predicted_classes_mnist = np.argmax(predictions_mnist, axis=1)
print(predicted_classes_mnist)


Epoch 1/5
1875/1875 [==============================] - 61s 32ms/step - loss: 0.1482 - accuracy: 0.9541 - val_loss: 0.0524 - val_accuracy: 0.9827
Epoch 2/5
1875/1875 [==============================] - 59s 31ms/step - loss: 0.0478 - accuracy: 0.9854 - val_loss: 0.0508 - val_accuracy: 0.9828
Epoch 3/5
1875/1875 [==============================] - 59s 31ms/step - loss: 0.0340 - accuracy: 0.9891 - val_loss: 0.0289 - val_accuracy: 0.9908
Epoch 4/5
1875/1875 [==============================] - 57s 30ms/step - loss: 0.0254 - accuracy: 0.9921 - val_loss: 0.0678 - val_accuracy: 0.9794
Epoch 5/5
313/313 [==============================] - 3s 9ms/step - loss: 0.0272 - accuracy: 0.9915

Test Loss: 0.0272, 
Test Accuracy: 0.9915
313/313 [==============================] - 4s 13ms/step
[7 2 1 ... 4 5 6]


[Problem 7] Rewriting to PyTorch
## 1. Iris Binary Classification with PyTorch

In [36]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the Iris dataset
iris = pd.read_csv('Iris.csv')

# Filter for Iris-versicolor and Iris-virginica
iris_binary = iris[(iris['Species'] == 'Iris-versicolor') | (iris['Species'] == 'Iris-virginica')]

# Extract features and labels
X = iris_binary.drop('Species', axis=1).values
y = np.where(iris_binary['Species'] == 'Iris-versicolor', 0, 1)

# Split into training and testing sets
X_train_binary, X_test_binary, y_train_binary, y_test_binary = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_binary = scaler.fit_transform(X_train_binary)
X_test_binary = scaler.transform(X_test_binary)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_binary)
y_train_tensor = torch.FloatTensor(y_train_binary).unsqueeze(1)  # Add a dimension for binary classification
X_test_tensor = torch.FloatTensor(X_test_binary)
y_test_tensor = torch.FloatTensor(y_test_binary).unsqueeze(1)

# Define the model
class IrisBinaryModel(nn.Module):
    def __init__(self):
        super(IrisBinaryModel, self).__init__()
        self.fc1 = nn.Linear(4, 10)
        self.fc2 = nn.Linear(10, 10)
        self.fc3 = nn.Linear(10, 1)  # Output layer for binary classification

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

# Instantiate the model, define loss function and optimizer
model_binary = IrisBinaryModel()
loss_fn = nn.BCELoss()
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)